In [1]:
import cv2
import numpy as np

In [3]:
def f1(x):
    pass

# Window Sizes 
FRAME_H, FRAME_W = 430, 1280   # Main display window
SINGLE_W = FRAME_W // 2
CTRL_H, CTRL_W = 115, 420      # Control window size
font = cv2.FONT_HERSHEY_SIMPLEX

# Window Setup
cv2.namedWindow("HSV Lower", cv2.WINDOW_NORMAL)
cv2.resizeWindow("HSV Lower", CTRL_W, CTRL_H)

cv2.namedWindow("HSV Upper", cv2.WINDOW_NORMAL)
cv2.resizeWindow("HSV Upper", CTRL_W, CTRL_H)

cv2.namedWindow("RGB Controls", cv2.WINDOW_NORMAL)
cv2.resizeWindow("RGB Controls", CTRL_W, CTRL_H)

cv2.namedWindow("Changing Colour", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Changing Colour", FRAME_W, FRAME_H)

# Trackbars
# HSV Lower And Upper
cv2.createTrackbar("H Low", "HSV Lower", 90, 179, f1)
cv2.createTrackbar("S Low", "HSV Lower", 10, 255, f1)
cv2.createTrackbar("V Low", "HSV Lower", 20, 255, f1)
cv2.createTrackbar("H Up", "HSV Upper", 120, 179, f1)
cv2.createTrackbar("S Up", "HSV Upper", 255, 255, f1)
cv2.createTrackbar("V Up", "HSV Upper", 255, 255, f1)

# RGB
cv2.createTrackbar("R", "RGB Controls", 0, 255, f1)
cv2.createTrackbar("G", "RGB Controls", 0, 255, f1)
cv2.createTrackbar("B", "RGB Controls", 255, 255, f1)

vid = cv2.VideoCapture(0)
while True:
    suc,frame = vid.read()
    frame = cv2.flip(frame, 1)
    frame_small = cv2.resize(frame, (SINGLE_W, FRAME_H))
    hsv = cv2.cvtColor(frame_small, cv2.COLOR_BGR2HSV)

    # HSV controls
    h_low = cv2.getTrackbarPos("H Low", "HSV Lower")
    s_low = cv2.getTrackbarPos("S Low", "HSV Lower")
    v_low = cv2.getTrackbarPos("V Low", "HSV Lower")
    h_up = cv2.getTrackbarPos("H Up", "HSV Upper")
    s_up = cv2.getTrackbarPos("S Up", "HSV Upper")
    v_up = cv2.getTrackbarPos("V Up", "HSV Upper")

    # RGB value controllers
    r = cv2.getTrackbarPos("R", "RGB Controls")
    g = cv2.getTrackbarPos("G", "RGB Controls")
    b = cv2.getTrackbarPos("B", "RGB Controls")

    # Mask and recolor
    mask=cv2.inRange(hsv,np.array([h_low,s_low,v_low]),np.array([h_up,s_up,v_up]))
    mask = cv2.medianBlur(mask, 5)

    result_small = frame_small.copy()
    result_small[mask > 0] = [b, g, r]

    # Labels
    cv2.putText(frame_small, "Original Image", (10, 30), font, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(result_small, "Converted Image", (10, 30), font, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    combined = np.hstack((frame_small, result_small))
    cv2.imshow("Changing Colour", combined)

    # --- HSV Lower display ---
    hsv_lower_disp = np.zeros((CTRL_H, CTRL_W, 3), dtype=np.uint8)
    cv2.putText(hsv_lower_disp, f"H: {h_low}  S: {s_low}  V: {v_low}", (10, 70), font, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow("HSV Lower", hsv_lower_disp)

    # --- HSV Upper display ---
    hsv_upper_disp = np.zeros((CTRL_H, CTRL_W, 3), dtype=np.uint8)
    cv2.putText(hsv_upper_disp, f"H: {h_up}  S: {s_up}  V: {v_up}", (10, 70), font, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow("HSV Upper", hsv_upper_disp)

    # --- RGB display ---
    rgb_disp = np.zeros((CTRL_H, CTRL_W, 3), dtype=np.uint8)
    rgb_disp[:] = (b, g, r)
    cv2.putText(rgb_disp, f"R: {r}  G: {g}  B: {b}", (10, 70), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow("RGB Controls", rgb_disp)

    if cv2.waitKey(1) == ord('e'):
        break

vid.release()
cv2.destroyAllWindows()
